In [1]:
%pip install fhirpy mypy pydantic -U

In [2]:
from fhirpy import SyncFHIRClient

from generated.resources import Patient, HumanName

FHIR_SERVER_API = "<FHIR server URL>"
FHIR_SERVER_AUTH = "<Bearer, Basic or other supported auth mechanism>"

client = SyncFHIRClient(FHIR_SERVER_API, authorization=FHIR_SERVER_AUTH)

#### Define helper function to run mypy typechecking on jupyter cells

In [3]:
from mypy import api as typechecker

def typecheck(block: str):
    mypy_error, _, _ = typechecker.run([
        '--no-color-output',
        '--no-error-summary',
        '--ignore-missing-imports',
        '-c', "\n\n".join([In[2], block])
    ])
    return mypy_error or 'OK'

#### Describe `HumanName` resource with the generated resource model

In [4]:
name = HumanName(family="Smith", given=["Andrew"])

client.resources("Patient").search(name=name.family).fetch()

[]

#### Make sure the resource structure is aligned to the definition

In [5]:
typecheck(In[4])

'OK'

#### Should there be any errors in the constructor arguments both resource validation and typecheck would fail

In [6]:
patient = Patient(name=HumanName(family="Smith", given=["Andrew"]), gender="male")

ValidationError: 1 validation error for Patient
name
  value is not a valid list (type=type_error.list)

In [7]:
typecheck(In[6])

'<string>:10: error: Argument "name" to "Patient" has incompatible type "HumanName"; expected "Optional[List[HumanName]]"  [arg-type]\n'

#### Create fhir-py resource from the model

In [8]:
client.resource("Patient", **patient.model_dump(exclude_none=True)).save()

#### Type-cast fhir-py resource back to the definition to work with data fields

In [9]:
patient = Patient.model_validate(client.resources("Patient").search(name="Smith").first())
patient.name

[HumanName(id=None, use=None, text=None, given=['Andrew'], family='Smith', prefix=None, suffix=None, period=None, extension=None)]

In [10]:
typecheck(In[9])

'OK'